In [16]:
import matplotlib.pyplot as plt
import gymnasium as gym
from evaluate import evaluate_HIV, evaluate_HIV_population
from train import ProjectAgent  # Replace DummyAgent with your agent implementation
from statistics import mean
from functools import partial
from gymnasium.wrappers import TimeLimit
from env_hiv import HIVPatient
from interface import Agent
import random
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
import math
import random
import matplotlib
import time
from collections import namedtuple
from itertools import count
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
from torch.distributions import Categorical
import ffmpeg
import seaborn as sns
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
env=TimeLimit(HIVPatient(), max_episode_steps=200)
'''nb_episodes = 100
states = np.zeros((nb_episodes*200,env.observation_space.shape[0]))
for i in range(nb_episodes):
    s,_ = env.reset()
    for j in range(200):
        s2,r,d,trunc,_ = env.step(env.action_space.sample())
        states[i*200+j] = s2
    print(i, end=', ')

states_means = states[:58*200].mean(axis=0)
states_stds = states[:58*200].std(axis=0)
#save them
np.save('states_means.npy', states_means)
np.save('states_stds.npy', states_stds)
#load them
states_means = np.load('states_means.npy')
states_stds = np.load('states_stds.npy')''';

In [ ]:
from dqn_agent import ReplayBuffer, greedy_action, evaluate_agent, dqn_agent

# DQN

## Train from scratch

In [36]:
env = TimeLimit(HIVPatient(), max_episode_steps=200)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Declare network
state_dim = env.observation_space.shape[0]
n_action = env.action_space.n 
nb_neurons=128
DQN = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(), 
                          nn.Linear(nb_neurons, n_action)).to(device)

# DQN config
config = {'nb_actions': n_action, 'learning_rate': 0.01,
          'gamma': 0.95, 'buffer_size': 100_000,
          'epsilon_min': 0.01, 'epsilon_max': 1.,
          'epsilon_decay_period': 5_000, 'epsilon_delay_decay': 600,
          'batch_size': 32, 'model_path': 'best_model_standard_scaling.pth',
          'scaling': 'standard', 'criterion': torch.nn.SmoothL1Loss()}

# Train agent
agent = dqn_agent(config, DQN)
scores = agent.train(env, 300)
#plt.plot(scores)

Episode   1, epsilon   1.00, batch size   200, episode return 10.3M, loss 39189.9
Evaluation deterministic reward 6.8M
Episode   2, epsilon   1.00, batch size   400, episode return 11.0M, loss 430363.0
Episode   3, epsilon   1.00, batch size   600, episode return  9.2M, loss 1643426.2
Episode   4, epsilon   0.96, batch size   800, episode return 10.4M, loss 3523872.8
Episode   5, epsilon   0.92, batch size  1000, episode return  9.1M, loss 4551970.0
Episode   6, epsilon   0.88, batch size  1200, episode return  8.8M, loss 5777839.0
Episode   7, epsilon   0.84, batch size  1400, episode return  8.9M, loss 3334394.5
Episode   8, epsilon   0.80, batch size  1600, episode return  7.7M, loss 2015005.1
Episode   9, epsilon   0.76, batch size  1800, episode return 14.4M, loss 1441623.5
Episode  10, epsilon   0.72, batch size  2000, episode return 16.5M, loss 1683770.2
Episode  11, epsilon   0.68, batch size  2200, episode return  8.9M, loss 819838.5
Episode  12, epsilon   0.64, batch size  24

## Train a pre-trained model

In [33]:
env = TimeLimit(HIVPatient(), max_episode_steps=200)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Declare network
state_dim = env.observation_space.shape[0]
n_action = env.action_space.n 
nb_neurons=128
config = {'nb_actions': n_action, 'learning_rate': 0.01,
          'gamma': 0.95, 'buffer_size': 100_000,
          'epsilon_min': 0.01, 'epsilon_max': 1,
          'epsilon_decay_period': 5_000, 'epsilon_delay_decay': 1,
          'batch_size': 32, 'model_path': 'best_model_standard_scaling.pth',
          'scaling': 'standard', 'criterion': torch.nn.SmoothL1Loss()}

##Train and load the best model
DQN = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(), 
                          nn.Linear(nb_neurons, n_action)).to(device)


#load the best model
DQN.load_state_dict(torch.load('best_model_standard_scaling.pth', weights_only=True))
agent = dqn_agent(config, DQN, load=True)
scores = agent.train(env, 500)

Loaded model with deterministic reward 5819.66M
Episode   1, epsilon   0.96, batch size   200, episode return 10.6M, loss 21571418.0
Episode   2, epsilon   0.92, batch size   400, episode return  7.9M, loss 1540596.8
Episode   3, epsilon   0.88, batch size   600, episode return 12.2M, loss 287791.9
Episode   4, epsilon   0.84, batch size   800, episode return  6.9M, loss 128160.9
Episode   5, epsilon   0.80, batch size  1000, episode return 13.0M, loss 120558.4
Episode   6, epsilon   0.76, batch size  1200, episode return  9.3M, loss 62426.2
Episode   7, epsilon   0.72, batch size  1400, episode return  9.8M, loss 55130.7
Episode   8, epsilon   0.68, batch size  1600, episode return 11.9M, loss 37057.6
Episode   9, epsilon   0.64, batch size  1800, episode return  9.2M, loss 34196.6
Episode  10, epsilon   0.60, batch size  2000, episode return  8.1M, loss 28444.3
Episode  11, epsilon   0.56, batch size  2200, episode return  8.7M, loss 19986.3
Episode  12, epsilon   0.53, batch size  2

# Evaluate Agent

In [35]:
from evaluate import evaluate_HIV, evaluate_HIV_population
from train import ProjectAgent  # Replace DummyAgent with your agent implementation

agent = ProjectAgent()
agent.load()
# Evaluate agent and write score.
score_agent: float = evaluate_HIV(agent=agent, nb_episode=5)
score_agent_dr: float = evaluate_HIV_population(agent=agent, nb_episode=20)
print(f"Score agent: {score_agent/1_000_000:.2f}M")
print(f"Score agent DR: {score_agent_dr/1_000_000:.2f}M")

Score agent: 7655.63M
Score agent DR: 1398.73M


# ActorCritic

In [36]:
from actor_critic import Actor, Critic, ActorCritic, evaluate_actor

In [55]:
# Direct variable assignments instead of argument parsing
class Args:
     def __init__(self, device="cpu", load=False, plot=True, model='reinforce_hiv/model.pt', 
                  lr=0.01, episodes=1000, gamma=0.95, scaling='standard'): 
          self.device = device  # cuda device
          self.load = load  # if loading an existing model
          self.plot = plot  # if plotting an existing model
          self.model = model  # model path
          self.lr = lr  # learning rate
          self.episodes = episodes  # number of episodes
          self.gamma = gamma  # discount factor
          self.scaling = scaling  # scaling method

class Runner(): 
     def __init__(self, env, logs="hiv", args=Args()):
          self.device = args.device
          self.actor = Actor(env.observation_space.shape[0], env.action_space.n).to(self.device)
          self.critic = Critic(env.observation_space.shape[0]).to(self.device)
          self.a_opt = optim.Adam(self.actor.parameters(), lr=args.lr)
          self.c_opt = optim.Adam(self.critic.parameters(), lr=args.lr)
          self.gamma = args.gamma
          self.logs = logs
          self.writer = SummaryWriter(logs)
          self.entropy = 0
          self.plots = {"Actor Loss": [], "Critic Loss": [], "Reward": [], "Mean Reward": []}
          self.load = args.load
          self.env = env
          self.states_means = np.load('states_means.npy')
          self.states_stds = np.load('states_stds.npy')
          self.scaling = args.scaling
          self.criterion = torch.nn.SmoothL1Loss()
          if args.load: 
               ac = ActorCritic(self.actor, self.critic)
               ac.load_state_dict(torch.load(args.model))
               self.actor = ac.actor
               self.critic = ac.critic

     def env_step(self, action):
          state, reward, done, trunc, log = self.env.step(action)
          return torch.FloatTensor([state]).to(self.device), torch.FloatTensor([reward]).to(self.device), done, trunc, log

     def select_action(self, state):
          # Convert state to tensor
          probs = self.actor(state)
          c = Categorical(probs)
          action = c.sample()

          # Place log probabilities into the policy history log\pi(a | s)
          if self.actor.policy_history.dim() != 0: 
                self.actor.policy_history = torch.cat([self.actor.policy_history, c.log_prob(action)])
          else: 
                self.actor.policy_history = (c.log_prob(action))
     
          return action
  
     def estimate_value(self, state): 
          pred = self.critic(state).squeeze(0)
          if self.critic.value_history.dim() != 0: 
               self.critic.value_history = torch.cat([self.critic.value_history, pred])
          else: 
               self.critic.policy_history = (pred)

     def update_a2c(self):
          R = 0
          q_vals = []

          # "Unroll" the rewards, apply gamma
          for r in self.actor.reward_episode[::-1]: 
               R = r + self.gamma * R
               q_vals.insert(0, R)
     
          q_vals = torch.FloatTensor(q_vals).to(self.device)
          values = self.critic.value_history
          log_probs = self.actor.policy_history
     
          advantage = q_vals - values
  
          self.c_opt.zero_grad()
          critic_loss = 0.05*self.criterion(values, q_vals)
          critic_loss.backward()
          self.c_opt.step()

          self.a_opt.zero_grad()
          actor_loss = -(log_probs * advantage.detach()).mean() + 0.0001 * self.entropy
          actor_loss.backward()
          self.a_opt.step()

          self.actor.reward_episode = []
          self.actor.policy_history = Variable(torch.Tensor()).to(self.device)
          self.critic.value_history = Variable(torch.Tensor()).to(self.device)
     
          return actor_loss, critic_loss
  
     def train(self, episodes=200, smooth=10):
          smoothed_reward = []
          best_deterministic_reward = -np.inf
          if self.load:
            best_deterministic_reward = evaluate_actor(self.actor, self.env, 1, scaling=self.scaling)/1_000_000
            print(f"Loaded model with deterministic reward {best_deterministic_reward:.2f}M")
          for episode in range(episodes): 
                rewards = 0
                state = self.env.reset()[0]
                state = (state - self.states_means) / self.states_stds
                self.entropy = 0
                done = False
                true_reward = 0

                for step in range(200): 
                    self.estimate_value(state)
          
                    policy = self.actor(state).cpu().detach().numpy()
                    action = self.select_action(state)
                    e = -np.sum(np.mean(policy) * np.log(policy))
                    self.entropy += e

                    state, reward, done, trunc, _ = self.env.step(action.data[0].item())
                    state = (state - self.states_means) / self.states_stds
                    true_reward += reward/1_000_000
                    reward= np.log10(reward + 100_000)
                    rewards += reward

                    self.actor.reward_episode.append(reward)

                    if done or trunc:
                         break
                deterministic_reward = evaluate_actor(self.actor, self.env, 1, scaling=self.scaling)/1_000_000
                if deterministic_reward > best_deterministic_reward:
                    best_deterministic_reward = deterministic_reward
                    ac = ActorCritic(self.actor, self.critic)
                    torch.save(ac.state_dict(), '%s/model_a2c.pt' % self.logs)
                    torch.save(best_deterministic_reward, self.logs + '/best_deterministic_reward_a2c.pth')
                    print("Evaluation deterministic reward", f"{deterministic_reward:.1f}M")

                smoothed_reward.append(true_reward)
                if len(smoothed_reward) > smooth: 
                    smoothed_reward = smoothed_reward[-1*smooth: -1]

                a_loss, c_loss = self.update_a2c()
        
                self.writer.add_scalar("Critic Loss", c_loss, episode)
                self.writer.add_scalar("Actor Loss", a_loss, episode)
                self.writer.add_scalar("Reward", true_reward, episode)
                self.writer.add_scalar("Mean Reward", np.mean(smoothed_reward), episode)

                self.plots["Critic Loss"].append(c_loss.item() * 100)
                self.plots["Actor Loss"].append(a_loss.item())
                self.plots["Reward"].append(true_reward)
                self.plots["Mean Reward"].append(np.mean(smoothed_reward))

                if episode % 1 == 0: 
                    print("\tEpisode {} \t Final Reward {:.2f}M \t Average Reward: {:.2f}M".format(episode, true_reward, np.mean(smoothed_reward)),
                          "Actor Loss: ", a_loss.item(), "Critic Loss: ", c_loss.item())

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = TimeLimit(HIVPatient(), max_episode_steps=200)

args = Args(device=device, load=False, 
          plot=True, model='reinforce_hiv/model.pt', 
          lr=0.01, episodes=500, gamma=0.99, scaling='standard')

runner = Runner(env, logs="reinforce_hiv", args=args)

print("[Train]\tTraining Beginning ...")
runner.train(args.episodes)
print("[End]\tDone. Congratulations!")

[Train]	Training Beginning ...
Evaluation deterministic reward 3.1M
	Episode 0 	 Final Reward 9.40M 	 Average Reward: 9.40M Actor Loss:  401.8038635253906 Critic Loss:  14.735630989074707
Evaluation deterministic reward 5.2M
	Episode 1 	 Final Reward 8.83M 	 Average Reward: 9.11M Actor Loss:  353.3734130859375 Critic Loss:  14.66199779510498
	Episode 2 	 Final Reward 6.07M 	 Average Reward: 8.10M Actor Loss:  295.3632507324219 Critic Loss:  14.434295654296875
	Episode 3 	 Final Reward 6.16M 	 Average Reward: 7.61M Actor Loss:  258.5687561035156 Critic Loss:  14.362095832824707
	Episode 4 	 Final Reward 5.50M 	 Average Reward: 7.19M Actor Loss:  234.49424743652344 Critic Loss:  14.18640422821045
	Episode 5 	 Final Reward 4.38M 	 Average Reward: 6.72M Actor Loss:  196.78842163085938 Critic Loss:  13.936531066894531
	Episode 6 	 Final Reward 3.91M 	 Average Reward: 6.32M Actor Loss:  181.99671936035156 Critic Loss:  13.653963088989258
	Episode 7 	 Final Reward 3.79M 	 Average Reward: 6.01

KeyboardInterrupt: 

In [96]:
evaluate_HIV = partial(evaluate_actor, env=env)
score_agent: float = evaluate_HIV(agent=actor, nb_episode=1)
score_agent

np.float64(5426976.930085338)